In [ ]:
%run "./initialize"

# Create Schemas and Tables

In [2]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {staging_schema}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {bronze_schema}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {silver_schema}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_schema}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {staging_schema}.{staging_volume}")

In [ ]:
import pyspark.sql.functions as F

# DELETE DATA FROM VOLUME IF EXISTS

In [ ]:
import time

def wait_for_deletion(path, max_wait=60, sleep_interval=2):
    """
    Wait until the specified path is deleted or timeout occurs.

    :param path: Path to check for existence
    :param max_wait: Maximum wait time in seconds
    :param sleep_interval: Time to wait between checks
    """
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
      try:
        dbutils.fs.ls(path)
        time.sleep(sleep_interval)
      except Exception as e:
        if "java.io.FileNotFoundException" in str(e):
          print(f"Deletion confirmed: {path} does not exist.")
          return True
        else:
          raise e
    
    print(f"Warning: Timeout reached while waiting for {path} to be deleted.")
    return False

# Trigger deletion
dbutils.fs.rm(f"{volume_root_file_path}", True)

# Wait until fully deleted
wait_for_deletion(volume_root_file_path)

# SUPPORTING TABLES

In [ ]:
region_df = (
  spark.sql(f"SELECT * FROM {sample_source_schema}.region").withColumn("load_timestamp", F.current_timestamp())
  .write.format("csv").mode("overwrite").options(**writer_options)
  .save(f"{volume_root_file_path}/region/region_{current_time_str}.csv")
)

nation_items_df = (
  spark.sql(f"SELECT * FROM {sample_source_schema}.nation").withColumn("load_timestamp", F.current_timestamp())
  .write.format("csv").mode("overwrite").options(**writer_options)
  .save(f"{volume_root_file_path}/nation1/nation_{current_time_str}.csv")
)

# CUSTOMER TABLES

In [ ]:
customer_df = (
    spark.sql(f"SELECT c_custkey as customer_id, c_name as name, c_acctbal as acctbal, c_mktsegment as mktseg FROM {sample_source_schema}.customer").dropDuplicates().withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/customer/customer_{current_time_str}.csv")
)

customer_address_df = (
    spark.sql(f"SELECT c_custkey as customer_id, c_address as address, c_nationkey as nat_id FROM {sample_source_schema}.customer").dropDuplicates().withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/customer_address/customer_address_{current_time_str}.csv")
)

customer_phone_df = (
    spark.sql(f"SELECT c_custkey as customer_id, 'M' as type, c_phone as phone FROM {sample_source_schema}.customer").dropDuplicates().withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/customer_phone/customer_phone{current_time_str}.csv")
)

# SUPPLIER TABLES

In [ ]:
supplier_df = (
    spark.sql(f"SELECT s_suppkey as supplier_id, s_name as name, s_acctbal as acctbal, s_comment as comment FROM {sample_source_schema}.supplier").dropDuplicates().withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/supplier/supplier_{current_time_str}.csv")
)

supplier_address_df = (
    spark.sql(f"SELECT s_suppkey as supplier_id, s_address as address, s_nationkey as nat_id FROM {sample_source_schema}.supplier").dropDuplicates().withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/supplier_address/supplier_address_{current_time_str}.csv")
)

supplier_phone_df = (
    spark.sql(f"SELECT s_suppkey as supplier_id, s_phone as phone FROM {sample_source_schema}.supplier").dropDuplicates().withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/supplier_phone/supplier_phone_{current_time_str}.csv")
)

# ORDERS TABLES

In [ ]:
orders_df = (
  spark.sql(f"SELECT * FROM {sample_source_schema}.orders").withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/orders/orders_{current_time_str}.csv")
)

line_items_df = (
  spark.sql(f"SELECT * FROM {sample_source_schema}.lineitem").withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/lineitem/lineitem_{current_time_str}.csv")
)

# PART TABLES

In [ ]:
part_df = (
  spark.sql(f"SELECT * FROM {sample_source_schema}.part").withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/part/part_{current_time_str}.csv")
)

partsupp_df = (
  spark.sql(f"SELECT * FROM {sample_source_schema}.partsupp").withColumn("load_timestamp", F.current_timestamp())
    .write.format("csv").mode("overwrite").options(**writer_options)
    .save(f"{volume_root_file_path}/partssupp/partssupp_{current_time_str}.csv")
)